In [2]:
%load_ext autoreload
%autoreload 2

from aind_auto_train import setup_logging
setup_logging()

# An open-loop simulation on old data from Janelia rigs

Create a `DynamicForagingAutoTrainManager` instance by connecting `df_behavior` and `df_manager` on s3.

In [3]:
from aind_auto_train.auto_train_manager import DynamicForagingAutoTrainManager
demo_manager_Janelia = DynamicForagingAutoTrainManager(manager_name='Janelia_demo',
                                         df_behavior_on_s3=dict(bucket='aind-behavior-data',
                                                                root='Han/ephys/report/all_sessions/export_all_nwb/',
                                                                file_name='df_sessions.pkl'),
                                         df_manager_root_on_s3=dict(bucket='aind-behavior-data',
                                                                    root='foraging_auto_training/'),
                                         if_rerun_all=False,
                                         )

2024-01-03 17:08:13 | INFO | aind_auto_train.util.aws_util | AWS credentials not found in environment variables. Try ~/.aws/credentials...
2024-01-03 17:08:13 | INFO | aind_auto_train.util.aws_util | Found AWS credential from ~/.aws/credentials!
2024-01-03 17:08:14 | INFO | aind_auto_train.util.aws_util | Dataframe imported from s3://aind-behavior-data/Han/ephys/report/all_sessions/export_all_nwb/df_sessions.pkl, len(df) = 4333
2024-01-03 17:08:14 | INFO | aind_auto_train.util.aws_util | Dataframe imported from s3://aind-behavior-data/foraging_auto_training/df_manager_Janelia_demo.pkl, len(df) = 3522
2024-01-03 17:08:14 | WARNING | aind_auto_train.auto_train_manager | current_stage_actual is not in df_behavior, we are in simulation mode!
2024-01-03 17:08:15 | INFO | aind_auto_train.util.aws_util | 17 objects downloaded from s3://aind-behavior-data/foraging_auto_training/saved_curriculums/ to C:\Users\admin/capsule/scratch/saved_curriculums/
2024-01-03 17:08:15 | INFO | aind_auto_train.

Here is the `df_behavior`, the same table shown in the [Streamlit app](https://foraging-behavior-browser.streamlit.app/)

In [3]:
demo_manager_Janelia.df_behavior

,level_0,index,session_date,h2o,session,subject_id,finished_trials,foraging_efficiency,photostim,task,...,valid_trial_end,valid_ratio,photostim_trial_ratio,headbar,age_in_weeks,session_length_in_hrs,relative_weight,water_total,water_per_trial_in_uL,curriculum_name
0,37,37,2019-09-11,HC16,1,447921,502,0.5020,,coupled_block_baiting,...,541.0,0.998152,NaN,janelia_headbar_at_janelia,33.142857,1.043928,1.000000,0.4,0.757576,coupled_block_baiting
1,36,36,2019-09-12,HC16,2,447921,581,0.5336,,coupled_block_baiting,...,587.0,0.991482,NaN,janelia_headbar_at_janelia,33.285714,1.029786,0.961722,1.1,3.536977,coupled_block_baiting
2,35,35,2019-09-13,HC16,3,447921,700,0.5888,,coupled_block_baiting,...,697.0,0.972106,NaN,janelia_headbar_at_janelia,33.428571,1.179464,1.009569,1.7,4.303797,coupled_block_baiting
3,34,34,2019-09-14,HC16,4,447921,333,0.7474,,coupled_block_baiting,...,343.0,0.968927,NaN,janelia_headbar_at_janelia,33.571429,0.732622,1.066986,1.1,4.641350,coupled_block_baiting
4,33,33,2019-09-15,HC16,5,447921,394,0.7258,,coupled_block_baiting,...,400.0,0.995025,NaN,janelia_headbar_at_janelia,33.714286,0.812617,1.090909,0.9,4.761905,coupled_block_baiting
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3652,4316,4316,2023-08-16,688237,17,688237,994,0.6768,,coupled_block_baiting,...,1022.0,1.000000,NaN,allen_headbar_at_allen,17.142857,1.770529,0.806036,1.2,0.751880,coupled_block_baiting
3653,4315,4315,2023-08-17,688237,18,688237,942,0.6862,,coupled_block_baiting,...,997.0,1.000000,NaN,allen_headbar_at_allen,17.285714,1.674205,0.806036,1.2,0.775194,coupled_block_baiting
3654,4314,4314,2023-08-18,688237,19,688237,969,0.6586,,coupled_block_baiting,...,1004.0,1.000000,NaN,allen_headbar_at_allen,17.428571,1.759870,0.817078,1.0,0.367647,coupled_block_baiting
3655,4313,4313,2023-08-21,688237,20,688237,916,0.6658,,coupled_block_baiting,...,950.0,0.985477,NaN,allen_headbar_at_allen,17.857143,1.693713,0.809717,1.1,0.778210,coupled_block_baiting


Now the `df_manager` table is empty.

In [4]:
demo_manager_Janelia.df_manager

,subject_id,session_date,task,session,session_at_current_stage,curriculum_name,task_schema_version,curriculum_version,curriculum_schema_version,curriculum_json_name,foraging_efficiency,finished_trials,metrics,current_stage_suggested,current_stage_actual,if_closed_loop,if_overriden_by_trainer,decision,next_stage_suggested


Calling `DynamicForagingAutoTrainManager.update()` will evaluate all "new" sessions that exist in `df_master` but not in `df_manager`, and give suggestions for the next training stage.

In [5]:
demo_manager_Janelia.update()

2024-01-03 16:51:11 | INFO | aind_auto_train.auto_train_manager | Found 3657 new sessions from 157 mice to evaluate
2024-01-03 16:51:11 | WARNING | aind_auto_train.auto_train_manager | No curriculum_version specified in df_behavior, use default curriculum "Coupled Baiting_curriculum_v0.1_schema_v0.2"
2024-01-03 16:51:11 | INFO | aind_auto_train.curriculum_manager | Loaded a DynamicForagingCurriculum model from 'Coupled Baiting_curriculum_v0.1_schema_v0.3.json'.
2024-01-03 16:51:11 | INFO | aind_auto_train.auto_train_manager | 447921, 2019-09-11, session 1: STAY at STAGE_1
2024-01-03 16:51:11 | WARNING | aind_auto_train.auto_train_manager | No curriculum_version specified in df_behavior, use default curriculum "Coupled Baiting_curriculum_v0.1_schema_v0.2"
2024-01-03 16:51:11 | INFO | aind_auto_train.curriculum_manager | Loaded a DynamicForagingCurriculum model from 'Coupled Baiting_curriculum_v0.1_schema_v0.3.json'.
2024-01-03 16:51:11 | INFO | aind_auto_train.auto_train_manager | 44792

Now `df_manager` is filled.

In [4]:
demo_manager_Janelia.df_manager[['subject_id', 'session_date', 
                                 'task', 'session', 'current_stage_suggested', 
                                 'current_stage_actual', 'decision', 'next_stage_suggested',
                                 'if_closed_loop', 'if_overriden_by_trainer'
                                 ]].query('subject_id == "473611"')

,subject_id,session_date,task,session,current_stage_suggested,current_stage_actual,decision,next_stage_suggested,if_closed_loop,if_overriden_by_trainer
788,473611,2020-08-11,Coupled Baiting,1,STAGE_1,STAGE_1,STAY,STAGE_1,False,False
789,473611,2020-08-12,Coupled Baiting,2,STAGE_1,STAGE_1,PROGRESS,STAGE_2,False,False
790,473611,2020-08-13,Coupled Baiting,3,STAGE_2,STAGE_2,PROGRESS,STAGE_3,False,False
791,473611,2020-08-14,Coupled Baiting,4,STAGE_3,STAGE_3,PROGRESS,STAGE_FINAL,False,False
792,473611,2020-08-15,Coupled Baiting,5,STAGE_FINAL,STAGE_FINAL,STAY,STAGE_FINAL,False,False
793,473611,2020-08-16,Coupled Baiting,6,STAGE_FINAL,STAGE_FINAL,STAY,STAGE_FINAL,False,False
794,473611,2020-08-17,Coupled Baiting,7,STAGE_FINAL,STAGE_FINAL,STAY,STAGE_FINAL,False,False
795,473611,2020-08-19,Coupled Baiting,8,STAGE_FINAL,STAGE_FINAL,ROLLBACK,STAGE_3,False,False
796,473611,2020-08-22,Coupled Baiting,9,STAGE_3,STAGE_3,ROLLBACK,STAGE_2,False,False
797,473611,2020-08-23,Coupled Baiting,10,STAGE_2,STAGE_2,STAY,STAGE_2,False,False


This demo is just an open-loop simulation, in the sense that `current_stage_actual` is not the same as `current_stage_suggested` in the table above -- I leave them NaN because those old mice did not even follow any pre-defined curriculum. But in reality, for each mouse, `df_manager` will be filled one session at a time, and our [python GUI](https://github.com/AllenNeuralDynamics/dynamic-foraging-task) should follow `current_stage_suggested` on the next session. Even if the suggestion is somehow overridden by the trainer, the `DynamicForagingAutoTrainManager` will know and use the transition rules of `current_stage_actual`.

Here are some simple stats

In [7]:
demo_manager_Janelia.df_manager_stats.query('subject_id == "473611"')

session_spent first_entry last_leave  \
subject_id current_stage_actual                                         
473611     STAGE_1                           2           1          2   
           STAGE_2                           3           3         11   
           STAGE_3                           3           4         12   
           STAGE_FINAL                       9           5         17   
           GRADUATED                         6          18         23   

                                session_spanned  n_STAY  n_PROGRESS  \
subject_id current_stage_actual                                       
473611     STAGE_1                            2       1           1   
           STAGE_2                            9       1           2   
           STAGE_3                            9       0           2   
           STAGE_FINAL                       13       7           1   
           GRADUATED                          6       6           0   

                                 n_ROLLBACK  
subject_id current_stage_actual              
473611     STAGE_1                        0  
           STAGE_2                        0  
           STAGE_3                        1  
           STAGE_FINAL                    1  
           GRADUATED                      0

And a simple plotly figure

In [8]:
fig = demo_manager_Janelia.plot_all_progress(if_show_fig=False, 
                                         x_axis='session',
                                         sort_by='progress_to_graduated',
                                         sort_order='descending')
fig.update_layout(height=2000)

# To render the plotly figure correctly in nbviewer
import plotly.io as pio
pio.renderers.default = "notebook_connected"

fig.show()

# Try new Bonsai sessions from 447

In [14]:
from aind_auto_train.auto_train_manager import DynamicForagingAutoTrainManager
demo_manager_447 = DynamicForagingAutoTrainManager(manager_name='447_demo',
                                     df_behavior_on_s3=dict(bucket='aind-behavior-data',
                                                            root='foraging_nwb_bonsai_processed/',
                                                            file_name='df_sessions.pkl'),
                                     df_manager_root_on_s3=dict(bucket='aind-behavior-data',
                                                                root='foraging_auto_training/'),
                                     if_rerun_all=False,
                                     )


2024-01-03 17:18:40 | INFO | aind_auto_train.util.aws_util | Dataframe imported from s3://aind-behavior-data/foraging_nwb_bonsai_processed/df_sessions.pkl, len(df) = 815
2024-01-03 17:18:40 | INFO | aind_auto_train.util.aws_util | Dataframe imported from s3://aind-behavior-data/foraging_auto_training/df_manager_447_demo.pkl, len(df) = 0
2024-01-03 17:18:40 | INFO | aind_auto_train.util.aws_util | 17 objects downloaded from s3://aind-behavior-data/foraging_auto_training/saved_curriculums/ to C:\Users\admin/capsule/scratch/saved_curriculums/
2024-01-03 17:18:40 | INFO | aind_auto_train.curriculum_manager | Found 3 curriculums in C:\Users\admin/capsule/scratch/saved_curriculums/


In [15]:
demo_manager_447.df_manager

,subject_id,session_date,task,session,session_at_current_stage,curriculum_name,task_schema_version,curriculum_version,curriculum_schema_version,curriculum_json_name,foraging_efficiency,finished_trials,metrics,current_stage_suggested,current_stage_actual,if_closed_loop,if_overriden_by_trainer,decision,next_stage_suggested


This time, since most of the sessions in `df_behvior` have been processed, only new sessions will be updated in `df_manager`. 

In [16]:
demo_manager_447.update()

2024-01-03 17:18:46 | INFO | aind_auto_train.auto_train_manager | Found 411 new sessions from 26 mice to evaluate
2024-01-03 17:18:46 | WARNING | aind_auto_train.auto_train_manager | current_stage_actual "None" is invalid for subject 00000, session 1.0, we are in open loop for this session.
2024-01-03 17:18:46 | INFO | aind_auto_train.curriculum_manager | Loaded a DynamicForagingCurriculum model from 'Coupled Baiting_curriculum_v0.1_schema_v0.3.json'.
2024-01-03 17:18:46 | INFO | aind_auto_train.auto_train_manager | 00000, 2023-09-19, session 1.0: STAY at STAGE_1
2024-01-03 17:18:46 | WARNING | aind_auto_train.auto_train_manager | current_stage_actual "None" is invalid for subject 641733, session 1.0, we are in open loop for this session.
2024-01-03 17:18:46 | INFO | aind_auto_train.curriculum_manager | Loaded a DynamicForagingCurriculum model from 'Coupled Baiting_curriculum_v0.1_schema_v0.3.json'.
2024-01-03 17:18:46 | INFO | aind_auto_train.auto_train_manager | 641733, 2023-11-08, s

In [18]:
demo_manager_447.df_manager[['subject_id', 'session_date', 
                                 'task', 'session', 'curriculum_name', 'current_stage_suggested', 
                                 'current_stage_actual', 'if_closed_loop', 'if_overriden_by_trainer', 'decision', 'next_stage_suggested'
                                 ]].query('subject_id == "689798"')

,subject_id,session_date,task,session,curriculum_name,current_stage_suggested,current_stage_actual,if_closed_loop,if_overriden_by_trainer,decision,next_stage_suggested
264,689798,2023-10-04,Coupled Baiting,1.0,Coupled Baiting,STAGE_1,STAGE_1,True,False,STAY,STAGE_1
265,689798,2023-10-05,Coupled Baiting,2.0,Coupled Baiting,STAGE_1,STAGE_FINAL,True,True,ROLLBACK,STAGE_3
266,689798,2023-10-06,Coupled Baiting,3.0,Coupled Baiting,STAGE_3,STAGE_3,True,False,STAY,STAGE_3
267,689798,2023-10-09,Coupled Baiting,4.0,Coupled Baiting,STAGE_3,STAGE_FINAL,True,True,ROLLBACK,STAGE_3
268,689798,2023-10-11,Coupled Baiting,5.0,Coupled Baiting,STAGE_3,STAGE_2,True,True,PROGRESS,STAGE_3
269,689798,2023-10-12,Coupled Baiting,6.0,Coupled Baiting,STAGE_3,STAGE_3,True,False,STAY,STAGE_3
270,689798,2023-10-13,Coupled Baiting,7.0,Coupled Baiting,STAGE_3,STAGE_1,True,True,PROGRESS,STAGE_2
271,689798,2023-10-16,Coupled Baiting,8.0,Coupled Baiting,STAGE_2,STAGE_FINAL,True,True,STAY,STAGE_FINAL
272,689798,2023-10-18,Coupled Baiting,9.0,Coupled Baiting,STAGE_FINAL,STAGE_3,True,True,STAY,STAGE_3
273,689798,2023-10-19,Coupled Baiting,10.0,Coupled Baiting,STAGE_3,STAGE_1,True,True,PROGRESS,STAGE_2


In [22]:
fig = demo_manager_447.plot_all_progress(if_show_fig=False)
fig.update_layout(height=500)

# To render the plotly figure correctly in nbviewer
import plotly.io as pio
pio.renderers.default = "notebook_connected"

fig.show()

In [23]:
fig = demo_manager_447.plot_all_progress(if_show_fig=False, x_axis='date')
fig.update_layout(height=500)

# To render the plotly figure correctly in nbviewer
import plotly.io as pio
pio.renderers.default = "notebook_connected"

fig.show()

In [24]:
fig = demo_manager_447.plot_all_progress(if_show_fig=False, 
                                         x_axis='date',
                                         sort_by='last_date',
                                         sort_order='descending')
fig.update_layout(height=500)

# To render the plotly figure correctly in nbviewer
import plotly.io as pio
pio.renderers.default = "notebook_connected"

fig.show()